 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 2: Optimisation sans contraintes
 ## Ulrich Baron-Fournier (2021196)
Tangi Migot et Paul Raynaud

In [2]:
using Pkg
Pkg.activate(".") #Accède au fichier Project.toml
Pkg.instantiate()
Pkg.status()

  Activating project at `c:\Users\Ulrizpascuit\Desktop\MTH8408\MTH8408-Hiv24\lab2`


Status `C:\Users\Ulrizpascuit\Desktop\MTH8408\MTH8408-Hiv24\lab2\Project.toml`
  [54578032] ADNLPModels v0.7.0
  [7073ff75] IJulia v1.24.2
  [40e66cde] LDLFactorizations v0.10.1
  [a4795742] NLPModels v0.20.0
  [5049e819] OptimizationProblems v0.7.3
  [91a5bcdd] Plots v1.39.0
  [37e2e46d] LinearAlgebra
  [de0858da] Printf
  [8dfed614] Test


In [3]:
using ADNLPModels, LinearAlgebra, NLPModels, Printf

In [4]:
# Problème test:
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1) # fonction objectif vue en classe
g(x) = 6 * [x[1]^2 - x[1] - 2*x[1]*x[2] + x[2]^2 + x[2]; -x[1]^2 + 2*x[1]*x[2] + x[1]] # le gradient de f
H(x) = 6 * [2*x[1]-1-2*x[2] -2*x[1]+2*x[2]+1; -2*x[1]+2*x[2]+1 2*x[1]] # la Hessienne de f

H (generic function with 1 method)

### Exercice 1: Newton avec recherche linéaire - amélioration du code

Ci-dessous, vous avez le code de deux fonctions qui ont été vues dans le cours, la recherche linéaire qui satisfait Armijo, et une méthode de Newton avec cette recherche linéaire. Le but de ce laboratoire est d'implémenter d'autres méthodes utiles pour résoudre des problèmes de grandes dimensions.

In [5]:
#Amélioration possibles: return also the value of f
function armijo(xk, dk, fk, gk, f)
  slope = dot(gk, dk) #doit être <0
  t = 1.0
  while f(xk + t * dk) > fk + 1.0e-4 * t * slope
    t /= 1.5
  end
  return t
end

armijo (generic function with 1 method)

In [6]:
function newton_armijo(f, g, H, x0; verbose::Bool = true)
  xk  = x0
  fk  = f(xk)
  gk = g(xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
    Hk = H(xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk = f(xk)
    gk = g(xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_armijo (generic function with 1 method)

In [7]:
x0 = [0.0, 0.0]
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1) # fonction objectif vue en classe
nlp = ADNLPModel(f, x0)

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  ForwardDiffADHessian,
  EmptyADbackend,
}
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                            

In [8]:
#SOLUTION: fonction à modifier
function newton_armijo(nlp, x0; verbose::Bool = true)
    t0 = time()
    xk  = x0
    fk  = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = gnorm0 = norm(gk)
    k = 0
    verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
    verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
    while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
      Hk = hess(nlp, xk)
      if neval_obj(nlp) > 1000
        println("évaluation de fonctions")
        break
      end
      dk = - Hk \ gk
      slope = dot(dk, gk)
      λ = 0.0
      α = 0
      while slope ≥ -1.0e-4 * norm(dk) * gnorm 
        if α < 5      
          λ = max(1.0e-3, 10 * λ)
          dk = - ((Hk + λ * I ) \ gk)
          slope = dot(dk, gk)
        else
          dk = -gk
          break
        end
        α += 1
      end
      if fk < -1e15
        println("problème non-borné")
        break
      end
      t = armijo(xk, dk, fk, gk, f)
      xk += t * dk
      fk = f(xk)
      gk = g(xk)
      gnorm = norm(gk)
      k += 1
      verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
      if time() - t0 > 60
        println("temps")
        break
      end
    end
    if k >= 100
      println("nombre d'itérations")
    end
    return xk
end

newton_armijo (generic function with 2 methods)

In [9]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
x0 = [0.5, 0.5]
nlp = ADNLPModel(f, x0)

newton_armijo(nlp, x0)

 k        fk ||∇f(x)||


 0  1.00e+00   4.5e+00


 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


2-element Vector{Float64}:
 2.323057366509693e-8
 2.3230573665124037e-8

### Exercice 2: LDLt-Newton avec recherche linéaire

On va maintenant modifier la méthode de Newton vu précédemment pour utiliser un package qui s'occupe de calculer une factorisation de la matrice hessienne tel que:
$$
\nabla^2 f(x) = LDL^T.
$$

In [10]:
using LDLFactorizations, LinearAlgebra

#### Utiliser cette technique pour calculer la direction de descente:

In [11]:
# Solution: modifier le calcul de la direction avec LDLFactorizations
function newton_ldlt_armijo(nlp, x0; verbose::Bool = true)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    Hk = Symmetric(triu(hess(nlp, xk)), :U)
    Sk = ldl_analyze(Hk)
    ldl_factorize!(Hk, Sk)
    dk = - Sk \ gk
    slope = dot(dk, gk)
    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_ldlt_armijo (generic function with 1 method)

In [12]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, [0.5, 0.5])

newton_ldlt_armijo(nlp, x0)

 k        fk ||∇f(x)||


 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


2-element Vector{Float64}:
 2.323057366509693e-8
 2.3230573665124037e-8

### Exercice 3: Méthode quasi-Newton: BFGS

In [13]:
# Solution: copier-coller votre newton_armijo ici et modifier le calcul de la direction avec la méthode de BFGS inverse skip.
function bfgs_quasi_newton_armijo(nlp, x0; verbose::Bool = true)
  xk = x0
  Hk = Matrix(1.0I, length(xk), length(xk))  
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
    k = 0
    while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
        dk = -Hk * gk  
        alpha = armijo(xk, dk, fk, gk, f)  
        sk = alpha*dk
        xk += sk
        fk_new = obj(nlp, xk)
        gk_new = grad(nlp, xk)
        yk = gk_new - gk
        if yk' * sk > 0
          ρk = 1 / dot(yk, sk)
          Hk = (I - ρk * sk * yk') * Hk * (I - ρk * yk * sk') + ρk * sk * sk'
        end
        fk, gk = fk_new, gk_new
        println("fk: ", fk)
        gnorm = norm(gk)
        k += 1
    end
    if fk < -1e15
      println("problème non-borné")
    end
    if k==100 
      println("iTER MAX")
    end
  return xk
end

bfgs_quasi_newton_armijo (generic function with 1 method)

In [14]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, [0.5, 0.5])

x = bfgs_quasi_newton_armijo(nlp, x0)

fk: 0.33333333333333315
fk: 0.20233196159122058
fk: -0.37708439613725964
fk: -0.7452682997168751
fk: -0.8574680944625642
fk: -0.9032163550497584
fk: -0.9643287052044303
fk: -0.9876660652933201
fk: -0.9999100038599352
fk: -0.9999995946309911
fk: -0.9999999989741873
fk: -0.9999999999999971


2-element Vector{Float64}:
 0.9999999947853835
 1.9103013141847714e-8

### Exercice 4: application à un problème de grande taille

In [17]:
using ADNLPModels, OptimizationProblems.ADNLPProblems, Test # Attention si vous ne faites pas using ADNLPModels avant ça ne fonctionne pas!

Si vous le souhaitez, il est possible d'accéder à certaines informations sur le problème en accédant à son meta:

In [19]:
using OptimizationProblems
OptimizationProblems.genrose_meta

Dict{Symbol, Any} with 17 entries:
  :has_equalities_only    => false
  :origin                 => :unknown
  :has_inequalities_only  => false
  :defined_everywhere     => missing
  :has_fixed_variables    => false
  :variable_ncon          => false
  :nvar                   => 100
  :is_feasible            => true
  :minimize               => true
  :ncon                   => 0
  :name                   => "genrose"
  :best_known_lower_bound => -Inf
  :objtype                => :other
  :best_known_upper_bound => 405.106
  :has_bounds             => false
  :variable_nvar          => true
  :contype                => :unconstrained

In [20]:
OptimizationProblems.meta

Row,nvar,variable_nvar,ncon,variable_ncon,minimize,name,has_equalities_only,has_inequalities_only,has_bounds,has_fixed_variables,objtype,contype,best_known_lower_bound,best_known_upper_bound,is_feasible,defined_everywhere,origin
,Int64,Bool,Int64,Bool,Bool,String,Bool,Bool,Bool,Bool,Symbol,Symbol,Real,Real,Bool?,Bool?,Symbol
1,1,false,0,false,true,AMPGO02,false,false,false,false,other,unconstrained,-Inf,0.839498,true,missing,unknown
2,1,false,0,false,true,AMPGO03,false,false,false,false,other,unconstrained,-Inf,2.88961,true,missing,unknown
3,1,false,0,false,true,AMPGO04,false,false,false,false,other,unconstrained,-Inf,-2.5666,true,missing,unknown
4,1,false,0,false,true,AMPGO05,false,false,false,false,other,unconstrained,-Inf,-0.0,true,missing,unknown
5,1,false,0,false,true,AMPGO06,false,false,false,false,other,unconstrained,-Inf,3.5177e-43,true,missing,unknown
6,1,false,0,false,true,AMPGO07,false,false,false,false,other,unconstrained,-Inf,2.56475,true,missing,unknown
7,1,false,0,false,true,AMPGO08,false,false,false,false,other,unconstrained,-Inf,-2.0928,true,missing,unknown
8,1,false,0,false,true,AMPGO09,false,false,false,false,other,unconstrained,-Inf,0.921136,true,missing,unknown
9,1,false,0,false,true,AMPGO10,false,false,false,false,other,unconstrained,-Inf,-0.0,true,missing,unknown


In [25]:
n = 100
model = genrose(n=n)
@test typeof(model) <: ADNLPModel

Test Passed

In [30]:
x0 = zeros(100)
bfgs_quasi_newton_armijo(model, x0)

fk: 40900.0
fk: 3412.0121633657614
fk: 4.222077144395051e16
fk: 1.083067512415147e10
fk: 7.999610635037326e35
fk: 1.7885943834776508e50
fk: 3.5428341847556473e108
fk: 2.1451377354665723e127
fk: 1.402139182529363e189
fk: Inf
fk: NaN


100-element Vector{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [31]:
n=10
model = tquartic(n=n)
@test typeof(model) <: ADNLPModel

Test Passed

In [32]:
x0 = rand(10)
bfgs_quasi_newton_armijo(model, x0)


fk: 5.209889753229934
fk: 20.01058943156945
fk: 3036.4413959056765
fk: 90.885219405152
fk: 686.094985824588
fk: 24351.389476734654
fk: 3.7545964987565406e7
fk: 1.2236707668399994e11
fk: 4.0367083132842706e14
fk: 7.857114334961082e26
fk: 5.7314450836621324e26
fk: 1.608810335631541e80
fk: 2.916391162004127e171
fk: 1.3085522524950529e252
fk: NaN


10-element Vector{Float64}:
 -1.1321887274464624e172
  1.4023808497985296e171
 -4.996726250463721e171
 -3.2154159859479532e171
 -3.7135790260243967e171
 -9.713235790884442e170
  2.9587865410600884e173
 -9.827398154235294e171
 -9.676439657242432e171
  0.4183267161844859